# Week 1 - Day 5 - Challenge

This is an implementation of three calls to GPT-4o-mini to create a brochure from a website and translate the brochure into French. The calls do the following:

*    The first call gets the related links from a conference website through multi-shot prompting.
*    The second call creates a brochure from the conference website using the related links from the first call and the information on the website.
*    The third call translates the brochure created by the second call from English to French.

## 0. Import Libraries

In [ ]:
import json
import os
import requests

from bs4 import BeautifulSoup
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from typing import Any, Dict, List

## 1. Load Environment Variables and Constants

In [ ]:
load_dotenv(override=True)
api = OpenAI()
CONFERENCE_NAME = "CCECE2025"
WEBSITE_URL = "https://ccece2025.ieee.ca/"
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("No API key was found!")
elif not api_key.startswith("sk-proj-"):
    print(
        "An API key was found, but it does not start with 'sk-proj-'! " + \
        "Please ensure you are using the right key."
        )
elif api_key.strip() != api_key:
    print(
        "An API key was found, but it looks like it might have space " + \
        "or tab characters at the start or end! Please remove them."
        )
else:
    print("API key found and looks good so far!")

## 2. Create a Class to Represent a Webpage

In [ ]:
class Website:
    """ """
    def __init__(
        self,
        url: str
        ) -> None:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " + \
            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 " + \
            "Safari/537.36"
            }
        self.url = url
        response = requests.get(
            url=url,
            headers=headers
            )
        self.body = response.content
        soup = BeautifulSoup(
            markup=self.body,
            features="html.parser"
            )
        self.title = soup.title.string if soup.title else "No title found!!!"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(
                separator="\n",
                strip=True
                )
        else:
            self.text = ""
        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link]

    def get_contents(self) -> str:
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
website = Website(url=WEBSITE_URL)

## 3. First Call to "gpt-4o-mini": Read Links on Webpage and Respond in Structured JSON

### 3.1. Create Prompts for First Call (Multi-Shot Prompting)

In [ ]:
first_call_system_prompt = """You are provided with a list of links found on \
a conference webpage. Your task is to decide which of the links would be most \
relevant to include in a brochure about the conference, such as links to \
committee page, authors page, or venue & travel page. You should respond in \
JSON format. Below are some examples.

Example 1:
Input:
[
    "https://example.com/about",
    "https://example.com/contact",
    "https://example.com/blog",
    "https://example.com/careers"
]
Output:
{
    "links": [
        {"type": "about page", "url": "https://example.com/about"},
        {"type": "careers page", "url": "https://example.com/careers"}
    ]
}

Example 2:
Input:
[
    "https://companysite.com/home",
    "https://companysite.com/jobs",
    "https://companysite.com/team",
    "https://companysite.com/privacy-policy"
]
Output:
{
    "links": [
        {"type": "careers page", "url": "https://companysite.com/jobs"},
        {"type": "about page", "url": "https://companysite.com/team"}
    ]
}

Example 3:
Input:
[
    "https://businesscorp.com/services",
    "https://businesscorp.com/company",
    "https://businesscorp.com/terms",
    "https://businesscorp.com/contact"
]
Output:
{
    "links": [
        {"type": "about page", "url": "https://businesscorp.com/company"}
    ]
}"""

In [ ]:
def get_links_user_prompt(website) -> str:
    user_prompt = ""
    user_prompt += "Below is the list of links on the website of "
    user_prompt += f"[{website.url}]. Please decide which of the links are "
    user_prompt += "relevant web links for a brochure about the conference. "
    user_prompt += "You should respond with the full https URL in JSON format "
    user_prompt += ". Do not include Terms of Service, Privacy, and email "
    user_prompt += "links."
    user_prompt += "\n\nLinks (some might be relevant links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
first_call_user_prompt = get_links_user_prompt(website=website)

### 3.2. Call "gpt-4o-mini" to Get Links

In [ ]:
def llm_get_links(
    api: OpenAI,
    system_prompt: str,
    user_prompt: str
    ) -> Dict[str, List[Dict[str, str]]]:
    response = api.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
        )
    return json.loads(response.choices[0].message.content)

In [ ]:
llm_links = llm_get_links(
    api=api,
    system_prompt=first_call_system_prompt,
    user_prompt=first_call_user_prompt
    )

## 4. Second Call to "gpt-4o-mini": Make Brochure

### 4.1. Create Prompts for Second Call

In [ ]:
second_call_system_prompt = ""
second_call_system_prompt += "You are an assistant that analyzes the contents "
second_call_system_prompt += "of several relevant pages from a conference "
second_call_system_prompt += "website and creates a short brochure about the "
second_call_system_prompt += "conference for academic researchers in formal "
second_call_system_prompt += "tone. Respond in markdown. Include details of "
second_call_system_prompt += "conference dates, call for papers, and "
second_call_system_prompt += "other opportunities if you have the information."

In [ ]:
def get_details_user_prompt(
    conference_name: str,
    website: Website,
    links: Dict[str, List[Dict[str, str]]]
    ) -> str:
    user_prompt = ""
    user_prompt += "You are looking at a conference webpage called "
    user_prompt += f"'{conference_name}'! Here are the contents of its "
    user_prompt += "landing page and other relevant pages. Use this "
    user_prompt += "information to build a short brochure of the conference "
    user_prompt += "in markdown.\n\n"
    user_prompt += website.get_contents()
    for link in links["links"]:
        user_prompt += f"\n\n{link["type"]}\n"
        user_prompt += Website(link["url"]).get_contents()
    return user_prompt[0:5000]  # Truncate if more than 5,000 characters

In [ ]:
second_call_user_prompt = get_details_user_prompt(
    conference_name=CONFERENCE_NAME,
    website=website,
    links=llm_links
    )

### 4.2. Call "gpt-4o-mini" to Create Brochure

In [ ]:
def llm_create_brochure(
    api: OpenAI,
    system_prompt: str,
    user_prompt: str
    ) -> None:
    stream_response = api.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True
        )
    response = ""
    
    # Create and initialize an interactive display output
    display_handle = display(
        Markdown(""),
        display_id=True
        )
    
    for chunk in stream_response:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```", "").replace("markdown", "")
        update_display(
            obj=Markdown(response),
            display_id=display_handle.display_id
            )
    return response

In [ ]:
brochure_text = llm_create_brochure(
    api=api,
    system_prompt=second_call_system_prompt,
    user_prompt=second_call_user_prompt
    )

## 5. Third Call to "gpt-4o-mini": Translate Entire Brochure to French

### 5.1. Create Prompts for Third Call

In [ ]:
third_call_system_prompt = """You are an assistant that analyzes the contents \
of a conference brochure and translates the brochure to French language while \
maintaining a formal tone suitable for academia. Respond in markdown."""

In [ ]:
def get_text_user_prompt(text: str) -> str:
    user_prompt = ""
    user_prompt += "The text below contains the contents of a conference "
    user_prompt += "brochure which you are required to translate to French "
    user_prompt += "language while maintaining a formal tone suitable for "
    user_prompt += "academia. Please provide answer in markdown. The brochure "
    user_prompt += f"text is as follows:\n\n{text}"
    return user_prompt

In [ ]:
third_call_user_prompt = get_text_user_prompt(text=brochure_text)

#### 5.2. Call "gpt-4o-mini" to Translate Brochure to French

In [ ]:
def llm_translate(
    api: OpenAI,
    system_prompt: str,
    user_prompt: str
    ) -> None:
    stream_response = api.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True
        )
    response = ""
    
    # Create and initialize an interactive display output
    display_handle = display(
        Markdown(""),
        display_id=True
        )
    
    for chunk in stream_response:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```", "").replace("markdown", "")
        update_display(
            obj=Markdown(response),
            display_id=display_handle.display_id
            )

    with open(file="w01_d05_conference_brochure_fr_output.txt", mode="w", encoding="utf-8") as f:
        f.write(response)

In [ ]:
llm_translate(
    api=api,
    system_prompt=third_call_system_prompt,
    user_prompt=third_call_user_prompt
    )

## 6. Final Output

# Conférence Canadienne en Génie Électrique et Informatique 2025 (CCECE 2025)

**Dates de la Conférence :**  
**26 - 29 Mai 2025**  
**Lieu :**  
Université de la Colombie-Britannique, Vancouver, Canada  

---

## Présentation de la Conférence

La Conférence Canadienne de l'IEEE en Génie Électrique et Informatique (CCECE 2025) constitue un rassemblement de premier plan pour les professionnels du secteur académique, de l'industrie et du gouvernement du monde entier afin de se réunir et d'échanger des idées sur les dernières avancées dans le domaine du génie électrique et informatique. Le thème de cette année, **« Par Terre, Mer et Air »**, met l'accent sur les technologies et solutions innovantes dans les systèmes marins et terrestres, avec une attention particulière sur :

- Capteurs Avancés
- Systèmes Autonomes
- Défis Environnementaux
- Transport Intelligent
- Transport Vert

---

## Appel à Contributions

### Soumettez Votre Travail

Les chercheurs sont invités à soumettre des articles sur divers sujets pertinents au génie électrique et informatique. Les détails clés de la soumission sont les suivants :

- **Format de l'Article :** 4-5 pages en format IEEE à 2 colonnes
- **Date Limite de Soumission des Articles :** 20 Janvier 2025
- **Notification d'Acceptation :** 3 Mars 2025
- **Soumission de l'Article Final :** 28 Mars 2025

D'autres opportunités incluent :

- **Propositions de Sessions Focalisées/Convocation :**  
  - Date Limite de Soumission : 6 Janvier 2025  
  - Notification d'Acceptation : 1 Février 2025  

- **Propositions de Tutoriels et Ateliers :**  
  - Date Limite de Soumission : 6 Janvier 2025  
  - Notification d'Acceptation : 1 Février 2025  

### Soumission des Résumés de Posters

- **Date Limite de Soumission :** 10 Mars 2025  
- **Soumission Finale du Résumé :** 28 Mars 2025  

---

## Informations sur l'Inscription

- **Date Limite d'Inscription Anticipée :** 28 Mars 2025  
- **Date Limite d'Inscription Régulière :** 21 Avril 2025  

---

## Opportunités de Participation

### Journées Thématiques Industrie

Ces journées spécialisées offriront des opportunités de développement professionnel à travers :

- Intervenants Pléniers
- Sessions de Panel
- Formations Pratiques

Une inscription flexible d'un jour sera disponible pour les professionnels de l'industrie.

### Tutoriels et Ateliers

Une gamme de tutoriels et d'ateliers se déroulera le premier jour de la conférence, **26 Mai 2025.**

### Exposition et Parrainage

Les entreprises cherchant à promouvoir leurs produits et services peuvent participer à l'exposition de la conférence. Pour toute demande concernant les opportunités d'exposition et de parrainage, veuillez contacter :

**Email :** ccece2025@ieee.ca

---

## Sujets d'Intérêt

La conférence couvrira un large éventail de sujets liés au génie électrique et informatique, y compris, mais sans s'y limiter :

- Communications, Réseautage et Traitement du Signal
- Apprentissage Automatique, Analyse de Données et Intelligence Artificielle
- Contrôle, Robotique et Systèmes Autonomes
- Transport Propre et Intelligent
- Éducation en Ingénierie

---

Rejoignez-nous pour faire avancer les frontières du génie électrique et informatique tout en vous connectant avec des experts de premier plan dans le domaine. Nous nous réjouissons de votre participation à CCECE 2025 !

Pour plus d'informations, visitez le [Site Web de CCECE 2025](#).